In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import dill
import numpy as np
import matplotlib.pyplot as plt
import scipy as scp
import os
from pathlib import Path
import consts as C
import ipywidgets as widgets

from os import listdir
from os.path import isfile, join
import datetime


results_dir = Path(os.getcwd()) / Path(C.Paths.RESULTS_DIR)
files = [f for f in listdir(results_dir) if isfile(join(results_dir, f))]
selected_files = []

file_selector = widgets.SelectMultiple(rows=10, options = files, description='Data files:', layout = widgets.Layout(width='fit-content'))
button = widgets.Button(description="Plot Selected")
ax = None
out = widgets.Output()

plt.rcParams['axes.grid'] = True


In [2]:
def draw_rate_vs_angle(data, filename):
    angle_to_rate = data["angle_to_rate"]
    name = data["conf"]["model_desc"]["name"]
    sound_key = data["conf"]["sound_key"]
    # cochlea = data["conf"]["cochlea_type"]
    
    angles = list(angle_to_rate.keys())

    arr_n_spikes_r_lso = [x["n_spikes_r_lso"] for angle, x in angle_to_rate.items()]
    arr_n_spikes_l_lso = [x["n_spikes_l_lso"] for angle, x in angle_to_rate.items()]
    arr_n_spikes_r_mso = [x["n_spikes_r_mso"] for angle, x in angle_to_rate.items()]
    arr_n_spikes_l_mso = [x["n_spikes_l_mso"] for angle, x in angle_to_rate.items()]
    
    fig, [lso, mso] = plt.subplots(2, 1)
    # fig.suptitle(f"{name} with {sound_key}")
    fig.suptitle(filename)
    plt.setp([lso,mso], xticks = angles)

    lso.plot(angles, arr_n_spikes_r_lso, ".-", label="right lso")
    lso.plot(angles, arr_n_spikes_l_lso, ".-", label="left lso")
    lso.set_ylabel("total spikes")
    _ = lso.legend()

    mso.plot(angles, arr_n_spikes_r_mso, ".-",  label="right mso")
    mso.plot(angles, arr_n_spikes_l_mso, ".-",  label="left mso")

    mso.set_xlabel("sound position")
    mso.set_ylabel("total spikes")
    _ = mso.legend()
    plt.tight_layout()
    plt.show()

In [3]:
@out.capture(clear_output=True,wait=True)
def on_button_clicked(b):
    global file_selector
    selected_files = file_selector.value
    for selected in selected_files:
        with open(join(results_dir,selected), "rb") as f:
            res = dill.load(f, ignore=True)
        draw_rate_vs_angle(res, selected)
    vbox.children = [file_selector,button, out]
        
button.on_click(on_button_clicked)
vbox = widgets.VBox([file_selector,button])
vbox

In [59]:
from utils.cochlea import sounds_to_spikes
from brian2hears import Sound, IRCAM_LISTEN
from brian2 import kHz,second
from consts import Paths

sound = Sound.tone(2*kHz, 1*second)
hrtfdb = IRCAM_LISTEN(Paths.IRCAM_DIR)
hrtfset = hrtfdb.load_subject(hrtfdb.subjects[0])
# far left
hrtf = hrtfset(azim=90, elev=0)
# binaural_sound = hrtf.filterbank(sound).process().T
binaural_sound = hrtf(sound)
# print(binaural_sound)
# print(binaural_sound.left)
left, right = binaural_sound.left, binaural_sound.right
print(left, right)

# sounds_to_spikes(binaural_sound,plot_spikes=True)

Sound duration 1. s, channels 1, samplerate 44.1 kHz Sound duration 1. s, channels 1, samplerate 44.1 kHz
